In [1]:
# import
import pandas as pd
import numpy as np
# import codecs
# import os
# import pymorphy2
# from string import ascii_lowercase, digits, whitespace
# from scipy.sparse import csr_matrix
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Импорт

In [2]:
#df_standards = pd.read_csv('../data/standards/df_standards_full_splitted.csv') #уже не надо
df_vacancies = pd.read_csv('df_vacancies.csv')
df_full = pd.read_csv("df_full.csv", sep=",", lineterminator='\n')
df_standards_prof = pd.read_csv("df_standards_prof.csv", lineterminator='\n')
full_text =pd.read_csv("full_text.csv")
samples_df = pd.read_csv('samples_df.csv')

In [135]:
df_standards_prof['full_processed_text'] = df_standards_prof['full_text'].apply(lambda text: process_text(str(text))['lemmatized_text_pos_tags'])

In [3]:
# df_vacancies = df_vacancies.dropna(subset=['text_item', 'type'])
# df_vacancies = df_vacancies.rename(index=str, columns={"text_item": "text"}) # уже сохранил
full_text = full_text.rename(index=str, columns={"nan_":"text"})
full_text = full_text.dropna(subset=['text'])
samples_df = samples_df.rename(index = str, columns={'vectors':'W2Vectors'})

In [46]:
tfidf = TfidfVectorizer()
# pickle.dump(tfidf.vocabulary_,open("vectors.pkl","wb"))

#vectors_full = tfidf.fit_transform(full_text['text'])

In [69]:
#df_standards_prof['vectors'] = tfidf.transform(df_standards_prof['processed_text'].apply(lambda x: ''.join(x))).todense().tolist()


In [147]:
print(tfidf.get_feature_names())

['000_', '004_', '00_', '020_', '02_', '08_', '09_', '0_', '0day_', '10', '1000', '1000_', '1000ть_grnd', '100_', '100к_noun', '101_', '104_', '10_', '10g_', '1101_', '115_', '11_', '11g_', '120_', '1240_', '126_', '12_', '12c_', '12g_', '12с_', '13_', '1427_', '14_', '150_', '152_', '153_', '15_', '16_', '178_', '17_', '18_', '19_', '1_', '1ad_', '1c_', '1podshipnik_', '1q_', '1tb_', '1x_', '1го_', '1год_noun', '1с8_', '1с_', '1сv8_', '1я_', '20', '20000_', '2000_', '200320072010_', '2003_', '2005_', '2006_', '2007_', '2008_', '2008r2_', '2008г_noun', '2009_', '200_', '200x_', '2010_', '2011_', '2012', '2012_', '2012r2_', '2012srv_', '2013_', '2014_', '2015_', '2016_', '2017_', '201_', '20_', '20чело_noun', '21_', '2200_', '223_', '2450_', '24_', '24x7_', '24x7x365_', '24х7_', '250_', '254_', '25_', '27001_', '27002_', '28_', '28xx_', '2900_', '2960_', '29xx_', '2_', '2d_', '2g_', '2is_', '2гр_', '2л_', '2х_', '3000_', '300_', '30_', '323_', '32_', '34_', '3500_', '35_', '35xx_', '360

In [48]:
#pickle.dump(tfidf.vocabulary_,open("vectors.pkl","wb"))

In [ ]:
#transformer = TfidfTransformer()
#loaded_vec = TfidfVectorizer(decode_error="replace",vocabulary=pickle.load(open("feature.pkl", "rb")))
#tfidf = transformer.fit_transform(loaded_vec.fit_transform(np.array)

In [161]:
full_text['vectors'] = tfidf.fit_transform(full_text.text).todense().tolist() #sparse
df_standards_prof['vectors'] = tfidf.transform(df_standards_prof['full_processed_text'].apply(lambda x: str(x))).todense().tolist()
samples_df['vectors'] = tfidf.transform(samples_df['processed_text'].apply(lambda x: ''.join(x))).todense().tolist()
df_vacancies['vectors'] = tfidf.transform(df_vacancies['processed_text'].apply(lambda x: ''.join(x))).todense().tolist()


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs
completed


In [162]:
print(full_text['vectors'].shape)
print(df_standards_prof['vectors'].shape)
print(df_vacancies['vectors'].shape)

(37027,)
(24395,)
(28593,)


In [163]:
print(len(full_text['vectors'][0]))
print(len(df_standards_prof['vectors'][0]))
print(len(df_vacancies['vectors'][0]))

10715
10715
10715


In [164]:
def most_similar(infer_vector, vectorized_corpus, topn = 5): #vector для сравнения, корпус для сравнения
    df_sim = vectorized_corpus
    vec = np.reshape(infer_vector,(1,-1))
    df_sim['sc'] = vectorized_corpus['vectors'].apply(lambda v : cosine_similarity(vec, np.reshape(v,(1,-1)))[0,0])
    df_sim = df_sim.sort_values(by='sc', ascending=False).head(n = topn)
    return df_sim

In [187]:
vac_sample = df_vacancies[df_vacancies['labels']=='10'].sample() #

infer_vector = vac_sample.iloc[0]['vectors'] #нужные вектора

ds = df_standards_prof[df_standards_prof['type'].isin(['activities', 'skills','knowledge'])]

In [188]:
print(vac_sample.index[0])
print(vac_sample.iloc[0]['name'])
print(vac_sample.iloc[0]['type'])
print(vac_sample.iloc[0]['text'])

9613
Системный администратор
duties
Обеспечивает функционирование ЛВС, серверов, Интернет, электронной почты, всех рабочих станций и программного обеспечения, оборудования, сохранность и достаточную степень конфиденциальности информации.


In [189]:
cosine_similarity(np.reshape(full_text.vectors[1],(1,-1)),np.reshape(full_text.vectors[0],(1,-1)))

array([[0.]])

In [190]:
similar_documents = most_similar(infer_vector, ds, topn = 5)

/home/mluser/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [191]:
similar_documents

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,kod_standard,num_gen_func,kod_gen_func,name_gen_func,gen_func_skill_level,kod_sim_func,name_sim_func,sim_func_skill_level,text,type,processed_text,vectors,full_text,full_processed_text,sc
17096,17096,17096,17096,6.026,26,F,Администрирование системного программного обес...,7,F/01.7,Установка системного программного обеспечения,7,Инсталляция программного обеспечения рабочих с...,activities,"['инсталляция_NOUN', 'программный_ADJF', 'обес...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Администрирование системного программного обес...,"[администрирование_NOUN, системный_ADJF, прогр...",0.250035
21661,21661,21661,21661,6.033,14,D,Разработка систем защиты информации автоматизи...,7,D/02.7,Разработка проектных решений по защите информа...,7,Классифицировать защищаемую информацию по вида...,skills,"['классифицировать_INFN', 'защищать_PRTF', 'ин...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Разработка систем защиты информации автоматизи...,"[разработка_NOUN, система_NOUN, защита_NOUN, и...",0.242320
21683,21683,21683,21683,6.033,17,E,Формирование требований к защите информации в ...,8,E/01.8,Обоснование необходимости защиты информации в ...,8,Классифицировать защищаемую информацию по вида...,skills,"['классифицировать_INFN', 'защищать_PRTF', 'ин...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Формирование требований к защите информации в ...,"[формирование_NOUN, требование_NOUN, защита_NO...",0.227703
20454,20454,20454,20454,6.031,13,D,Организационное управление в ИАС в защищенном ...,7,D/03.7,Организация работ по выполнению в ИАС требован...,7,Классифицировать защищаемую информацию по вида...,skills,"['классифицировать_INFN', 'защищать_PRTF', 'ин...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Организационное управление в ИАС в защищенном ...,"[организационный_ADJF, управление_NOUN, иас_, ...",0.210221
20416,20416,20416,20416,6.031,5,B,Проектирование ИАС в защищенном исполнении,7,B/03.7,Разработка проектных документов на создаваемые...,7,Классифицировать защищаемую информацию по вида...,skills,"['классифицировать_INFN', 'защищать_PRTF', 'ин...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Проектирование ИАС в защищенном исполнении Раз...,"[проектирование_NOUN, иас_, защитить_PRTF, исп...",0.207933


In [197]:
for idx, row in similar_documents.iterrows():
    print('kod_standard: ', row['kod_standard'],'type: ', row['type'],'  score = ',row['sc'])
    print(row['full_text'])
    print('================================')

kod_standard:  6.026 type:  activities   score =  0.2500354379322998
Администрирование системного программного обеспечения инфокоммуникационной системы организации Установка системного программного обеспечения Инсталляция программного обеспечения рабочих станций
kod_standard:  6.0329999999999995 type:  skills   score =  0.2423203679090272
Разработка систем защиты информации автоматизированных Разработка проектных решений по защите информации в автоматизированных системах Классифицировать защищаемую информацию по видам тайны и степеням конфиденциальности
kod_standard:  6.0329999999999995 type:  skills   score =  0.22770332076080052
Формирование требований к защите информации в автоматизированных системах Обоснование необходимости защиты информации в автоматизированной системе Классифицировать защищаемую информацию по видам тайны и степеням конфиденциальности и оценивать угрозы безопасности информации
kod_standard:  6.0310000000000015 type:  skills   score =  0.2102205710777926
Организац

In [193]:
print(vac_sample.index[0])
print(vac_sample.iloc[0]['name'])
print(vac_sample.iloc[0]['type'])
print(vac_sample.iloc[0]['text'])

9613
Системный администратор
duties
Обеспечивает функционирование ЛВС, серверов, Интернет, электронной почты, всех рабочих станций и программного обеспечения, оборудования, сохранность и достаточную степень конфиденциальности информации.


In [9]:
df_vacancies.head()

,Unnamed: 0,id,name,pred_labels_w2v,pred_labels_tfidf,labels,tag,text,type,processed_text,vectors
0,1112,22003685,Специалист ИТ,10,10,10,Обязанности:,"Установка, настройка программного обеспечения;",duties,"['установка_NOUN', 'настройка_NOUN', 'программ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1113,22003685,Специалист ИТ,10,10,10,Обязанности:,Работа с периферийными устройствами (профилакт...,duties,"['работа_NOUN', 'периферийный_ADJF', 'устройст...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1114,22003685,Специалист ИТ,10,10,10,Обязанности:,Прокладка компьютерных сетей;,duties,"['прокладка_NOUN', 'компьютерный_ADJF', 'сеть_...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1115,22003685,Специалист ИТ,10,10,10,Обязанности:,Администрирование серверной системы;,duties,"['администрирование_NOUN', 'серверный_ADJF', '...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1116,22003685,Специалист ИТ,10,10,10,Обязанности:,Диагностика неисправностей ИС.,duties,"['диагностика_NOUN', 'неисправность_NOUN', 'ис...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [88]:
df_vacancies['processed_text'].apply(lambda x: ''.join(x)).head()

0    ['установка_NOUN', 'настройка_NOUN', 'программ...
1    ['работа_NOUN', 'периферийный_ADJF', 'устройст...
2    ['прокладка_NOUN', 'компьютерный_ADJF', 'сеть_...
3    ['администрирование_NOUN', 'серверный_ADJF', '...
4    ['диагностика_NOUN', 'неисправность_NOUN', 'ис...
Name: processed_text, dtype: object

In [86]:
full_text = full_text.drop(columns = ['strvec'])

In [95]:
np.reshape(infer_vector,(1,-1))

array([[0., 0., 0., ..., 0., 0., 0.]])

In [129]:
import codecs
import os
import pymorphy2
from string import ascii_lowercase, digits, whitespace

morph = pymorphy2.MorphAnalyzer()

cyrillic = u"абвгдеёжзийклмнопрстуфхцчшщъыьэюя"

allowed_characters = ascii_lowercase + digits + cyrillic + whitespace

def complex_preprocess(text, additional_allowed_characters = "+#"):
    return ''.join([character if character in set(allowed_characters+additional_allowed_characters) else ' ' for character in text.lower()]).split()

def lemmatize(tokens, filter_pos):
    '''Produce normal forms for russion words using pymorphy2
    '''
    lemmas = []
    tagged_lemmas = []
    for token in tokens:
        parsed_token = morph.parse(token)[0]
        norm = parsed_token.normal_form
        pos = parsed_token.tag.POS        
        if pos is not None:
            if pos not in filter_pos:
                lemmas.append(norm)
                tagged_lemmas.append(norm + "_" + pos)
        else:
            lemmas.append(token)
            tagged_lemmas.append(token+"_")

    return lemmas, tagged_lemmas

def process_text(full_text, filter_pos=("PREP", "NPRO", "CONJ")):
    '''Process a single text and return a processed version
    '''
    single_line_text = full_text.replace('\n',' ')
    preprocessed_text = complex_preprocess(single_line_text)
    lemmatized_text, lemmatized_text_pos_tags = lemmatize(preprocessed_text, filter_pos=filter_pos)

    return { "full_text" : full_text,
    "single_line_text": single_line_text,
    "preprocessed_text": preprocessed_text,
    "lemmatized_text": lemmatized_text,
    "lemmatized_text_pos_tags": lemmatized_text_pos_tags}

In [130]:
df_standards_prof['full_text'] = df_standards_prof['name_gen_func'] + ' ' + df_standards_prof['name_sim_func'] + ' ' + df_standards_prof['text']

In [198]:
samples_df

,Unnamed: 0,id,name,pred_labels_w2v,pred_labels_tfidf,labels,tag,text,type,processed_text,W2Vectors,vectors
0,16302,24310970,Java-разработчик,NaN,NaN,14,Обязанности:,Построение процесса continuous integration/con...,duties,"['построение_NOUN', 'процесс_NOUN', 'continuou...",[ 1.07008163e-02 -6.16351999e-02 -1.42251953e-...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,24184,25620283,Программист,14,14,14,Чем нужно будет заниматься:,Разрабатывать/внедрять новые модули на основан...,duties,"['разрабатывать_INFN', 'внедрять_INFN', 'новое...",[ 1.05946764e-01 7.87466466e-02 -4.40551783e-...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,13619,23880141,Java-разработчик,14,14,14,Требования:,Опыт работы с СУБД (SQL).,requirements,"['опыт_NOUN', 'работа_NOUN', 'субд_NOUN', 'sql_']",[-0.05298915 0.05067005 -0.04726964 -0.006521...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,11843,23572369,Программист С++,"12,14","12,14",14,Требования:,"Знание технологий WinAPI, включая DLL, Windows...",requirements,"['знание_NOUN', 'технология_NOUN', 'winapi_', ...",[ 0.02964516 0.08149331 -0.02524044 0.017862...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,30247,27642679,Программист Delphi (MS SQL),14,"1,14",14,Требования:,опыт разработки многопоточных приложений на De...,requirements,"['опыт_NOUN', 'разработка_NOUN', 'многопоточны...",[-2.81809755e-02 5.56434169e-02 1.56999603e-...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,16473,24333420,Программист PHP ( на коробочную версию Битрикс...,"14,19",14,14,Обязанности:,Настройка интеграции с различными внутренними ...,duties,"['настройка_NOUN', 'интеграция_NOUN', 'различн...",[-7.15132001e-06 8.90142545e-02 -6.00658245e-...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,32297,28025416,Ведущий инженер-программист (бюро ТМС),14,14,14,Требования:,"Знание языков программирования C, С++.",requirements,"['знание_NOUN', 'язык_NOUN', 'программирование...",[-0.02775248 0.04711124 -0.01690753 0.030575...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,30148,27597093,Разработчик приложений для Андроид / Android D...,NaN,NaN,14,Обязанности,Участие в разработке и развитии наших продукто...,duties,"['участие_NOUN', 'разработка_NOUN', 'развитие_...",[-0.05152983 0.06037962 0.02488272 -0.009615...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,18037,24605056,Программист 1С,NaN,14,14,Требования:,"Умение работать в режиме многозадачности, само...",requirements,"['умение_NOUN', 'работать_INFN', 'режим_NOUN',...",[ 1.30881503e-01 -2.46298686e-02 3.40769477e-...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,13953,23949234,IOS разработчик,14,14,14,Обязанности:,Общение с заказчиком и командой Бекенд-програм...,duties,"['общение_NOUN', 'заказчик_NOUN', 'команда_NOU...",[ 1.44434469e-02 2.44132563e-01 -2.92823464e-...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [223]:
infer_vector_sample = samples_df.iloc[0] #нужные вектора
infer_vector = infer_vector_sample['vectors']

In [224]:
similar_documents = most_similar(infer_vector, ds, topn = 5)

/home/mluser/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [225]:
print(infer_vector_sample['text']+'\n')
for idx, row in similar_documents.iterrows():
    print('kod_standard: ', row['kod_standard'],'type: ', row['type'],'  score = ',row['sc'])
    print(row['full_text'])
    print('================================')

Построение процесса continuous integration/continuous delivery;

kod_standard:  6.035 type:  activities   score =  0.10249760451573246
Управление процессами и проектами по созданию (модификации) информационных ресурсов Управление процессом разработки программного обеспечения Планирование процесса разработки программного продукта
kod_standard:  6.0310000000000015 type:  knowledge   score =  0.10185903268034113
Применение ИАС в защищенном исполнении в процессах АИАД Автоматизированная информационно-аналитическая поддержка процессов принятия решений Структура, принципы построения и функционирования ситуационных центров
kod_standard:  6.0310000000000015 type:  knowledge   score =  0.10167018011284629
Применение ИАС в защищенном исполнении в процессах АИАД Автоматизированная информационно-аналитическая поддержка процессов принятия решений Основы построения и функционирования геоинформационных автоматизированных систем
kod_standard:  6.017 type:  activities   score =  0.10146643835611867
Орг

In [226]:
df_most_similar = pd.concat([df_most_similar,similar_documents])

In [395]:
def most_similar(infer_vector, vectorized_corpus, own = False, topn = 5): #vector для сравнения, корпус для сравнения
    if own == False:
        df_sim = vectorized_corpus
    else:
        df_sim = vectorized_corpus[vectorized_corpus['kod_standard'] == own]
    vec = np.reshape(infer_vector,(1,-1))
    df_sim['sc'] = df_sim['vectors'].apply(lambda v : cosine_similarity(vec, np.reshape(v,(1,-1)))[0,0])
    df_sim = df_sim.dropna()
    df_sim = df_sim.sort_values(by='sc', ascending=False).head(n = topn)
    return df_sim

In [382]:
df_standards_prof[df_standards_prof['kod_standard'] == '6.001']

/home/mluser/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py:1164: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


TypeError: invalid type comparison

In [400]:
def most_similar(infer_vector, vectorized_corpus, own, topn = 5): #vector для сравнения, корпус для сравнения
    if float(own) !=1:
        df_sim = vectorized_corpus[vectorized_corpus['kod_standard'] == float(own)]
    else:
        df_sim = vectorized_corpus
    vec = np.reshape(infer_vector,(1,-1))
    df_sim['sc'] = df_sim['vectors'].apply(lambda v : cosine_similarity(vec, np.reshape(v,(1,-1)))[0,0])
    df_sim = df_sim.dropna()
    df_sim = df_sim.sort_values(by='sc', ascending=False).head(n = topn)
    return df_sim

In [401]:
def similarity(vacancies, standards, own = 1):
    df_result = pd.DataFrame(columns = ['vac_text','prof_text','prof_type','prof_code','sc','vector_sim'])
    for index, sample in vacancies.iterrows():
        if own != 1:
            own = sample['kod_standard']
        similar_docs = most_similar(sample['vectors'],standards, own)[['full_text','type','kod_standard','sc']]
        similar_docs['vac_text'] = sample['text']
        similar_docs['name'] = sample['name']
        similar_docs['idx_vacancies'] = sample['Unnamed: 0']
        similar_docs['vector_sim'] = 'TfIdf'
        similar_docs['labels'] = sample['labels']
        similar_docs = similar_docs.rename(columns={
                 'full_text' : 'prof_text',
                 'type' : 'prof_type',
                
                 'kod_standard' : 'prof_code'
                 })   
        similar_docs['prof_name'] = sample['prof_name']

        df_result = pd.concat([df_result,similar_docs])
        print(index)
    return df_result

In [396]:
similar_samples = similarity(samples_df, df_standards_prof)
#similar_samples.to_csv('similar_samples.csv')

/home/mluser/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119


In [399]:
similar_samples.to_csv('similar_samples.csv')

In [406]:
similar_samples_own = similarity(samples_df,df_standards_prof, own = 0.1) 
#similar_samples_own.to_csv('similar_samples_own.csv')

/home/mluser/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/mluser/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119


In [407]:
similar_samples_own.to_csv('similar_samples_own.csv')

In [409]:
similar_samples_own

,idx_vacancies,labels,name,prof_code,prof_name,prof_text,prof_type,sc,vac_text,vector_sim
240,16302.0,14.0,Java-разработчик,6.001,Программист,Разработка требований и проектирование програм...,knowledge,0.042443,Построение процесса continuous integration/con...,TfIdf
150,16302.0,14.0,Java-разработчик,6.001,Программист,Проверка работоспособности и рефакторинг кода ...,skills,0.026375,Построение процесса continuous integration/con...,TfIdf
0,16302.0,14.0,Java-разработчик,6.001,Программист,Разработка и отладка программного кода Формали...,posts,0.000000,Построение процесса continuous integration/con...,TfIdf
168,16302.0,14.0,Java-разработчик,6.001,Программист,Интеграция программных модулей и компонент и в...,skills,0.000000,Построение процесса continuous integration/con...,TfIdf
156,16302.0,14.0,Java-разработчик,6.001,Программист,Проверка работоспособности и рефакторинг кода ...,skills,0.000000,Построение процесса continuous integration/con...,TfIdf
222,24184.0,14.0,Программист,6.001,Программист,Интеграция программных модулей и компонент и в...,knowledge,0.222457,Разрабатывать/внедрять новые модули на основан...,TfIdf
220,24184.0,14.0,Программист,6.001,Программист,Интеграция программных модулей и компонент и в...,knowledge,0.201002,Разрабатывать/внедрять новые модули на основан...,TfIdf
105,24184.0,14.0,Программист,6.001,Программист,Интеграция программных модулей и компонент и в...,activities,0.192685,Разрабатывать/внедрять новые модули на основан...,TfIdf
162,24184.0,14.0,Программист,6.001,Программист,Интеграция программных модулей и компонент и в...,skills,0.179778,Разрабатывать/внедрять новые модули на основан...,TfIdf
161,24184.0,14.0,Программист,6.001,Программист,Интеграция программных модулей и компонент и в...,skills,0.178624,Разрабатывать/внедрять новые модули на основан...,TfIdf


In [269]:
similar_samples.to_csv('similar_samples.csv')

In [288]:
similar_samples = pd.read_csv('similar_samples.csv')

In [391]:
similar_samples_own.columns

Index(['labels', 'name', 'prof_code', 'prof_name', 'prof_text', 'prof_type',
       'sc', 'vac_text', 'vector_sim'],
      dtype='object')

In [392]:
samples_df

,Unnamed: 0,id,name,pred_labels_w2v,pred_labels_tfidf,labels,tag,text,type,processed_text,W2Vectors,vectors,prof_name,kod_standard
0,16302,24310970,Java-разработчик,NaN,NaN,14,Обязанности:,Построение процесса continuous integration/con...,duties,"['построение_NOUN', 'процесс_NOUN', 'continuou...",[ 1.07008163e-02 -6.16351999e-02 -1.42251953e-...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Программист,6.001
1,24184,25620283,Программист,14,14,14,Чем нужно будет заниматься:,Разрабатывать/внедрять новые модули на основан...,duties,"['разрабатывать_INFN', 'внедрять_INFN', 'новое...",[ 1.05946764e-01 7.87466466e-02 -4.40551783e-...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Программист,6.001
2,13619,23880141,Java-разработчик,14,14,14,Требования:,Опыт работы с СУБД (SQL).,requirements,"['опыт_NOUN', 'работа_NOUN', 'субд_NOUN', 'sql_']",[-0.05298915 0.05067005 -0.04726964 -0.006521...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Программист,6.001
3,11843,23572369,Программист С++,"12,14","12,14",14,Требования:,"Знание технологий WinAPI, включая DLL, Windows...",requirements,"['знание_NOUN', 'технология_NOUN', 'winapi_', ...",[ 0.02964516 0.08149331 -0.02524044 0.017862...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Программист,6.001
4,30247,27642679,Программист Delphi (MS SQL),14,"1,14",14,Требования:,опыт разработки многопоточных приложений на De...,requirements,"['опыт_NOUN', 'разработка_NOUN', 'многопоточны...",[-2.81809755e-02 5.56434169e-02 1.56999603e-...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Программист,6.001
5,16473,24333420,Программист PHP ( на коробочную версию Битрикс...,"14,19",14,14,Обязанности:,Настройка интеграции с различными внутренними ...,duties,"['настройка_NOUN', 'интеграция_NOUN', 'различн...",[-7.15132001e-06 8.90142545e-02 -6.00658245e-...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Программист,6.001
6,32297,28025416,Ведущий инженер-программист (бюро ТМС),14,14,14,Требования:,"Знание языков программирования C, С++.",requirements,"['знание_NOUN', 'язык_NOUN', 'программирование...",[-0.02775248 0.04711124 -0.01690753 0.030575...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Программист,6.001
7,30148,27597093,Разработчик приложений для Андроид / Android D...,NaN,NaN,14,Обязанности,Участие в разработке и развитии наших продукто...,duties,"['участие_NOUN', 'разработка_NOUN', 'развитие_...",[-0.05152983 0.06037962 0.02488272 -0.009615...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Программист,6.001
8,18037,24605056,Программист 1С,NaN,14,14,Требования:,"Умение работать в режиме многозадачности, само...",requirements,"['умение_NOUN', 'работать_INFN', 'режим_NOUN',...",[ 1.30881503e-01 -2.46298686e-02 3.40769477e-...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Программист,6.001
9,13953,23949234,IOS разработчик,14,14,14,Обязанности:,Общение с заказчиком и командой Бекенд-програм...,duties,"['общение_NOUN', 'заказчик_NOUN', 'команда_NOU...",[ 1.44434469e-02 2.44132563e-01 -2.92823464e-...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Программист,6.001


In [299]:
df_standards_prof

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,kod_standard,num_gen_func,kod_gen_func,name_gen_func,gen_func_skill_level,kod_sim_func,name_sim_func,sim_func_skill_level,text,type,processed_text,vectors,full_text,full_processed_text,sc
0,0,0,0,6.001,0,A,Разработка и отладка программного кода,3,A/01.3,Формализация и алгоритмизация поставленных задач,3,Младший программист,posts,"['младший_ADJF', 'программист_NOUN']","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Разработка и отладка программного кода Формали...,"[разработка_NOUN, отладка_NOUN, программный_AD...",0.0
1,1,1,1,6.001,0,A,Разработка и отладка программного кода,3,A/01.3,Формализация и алгоритмизация поставленных задач,3,Техник-программист,posts,"['техника_NOUN', 'программист_NOUN']","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Разработка и отладка программного кода Формали...,"[разработка_NOUN, отладка_NOUN, программный_AD...",0.0
2,2,2,2,6.001,1,A,Разработка и отладка программного кода,3,A/02.3,Написание программного кода с использованием я...,3,Младший программист,posts,"['младший_ADJF', 'программист_NOUN']","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Разработка и отладка программного кода Написан...,"[разработка_NOUN, отладка_NOUN, программный_AD...",0.0
3,3,3,3,6.001,1,A,Разработка и отладка программного кода,3,A/02.3,Написание программного кода с использованием я...,3,Техник-программист,posts,"['техника_NOUN', 'программист_NOUN']","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Разработка и отладка программного кода Написан...,"[разработка_NOUN, отладка_NOUN, программный_AD...",0.0
4,4,4,4,6.001,2,A,Разработка и отладка программного кода,3,A/03.3,Оформление программного кода в соответствии с ...,3,Младший программист,posts,"['младший_ADJF', 'программист_NOUN']","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Разработка и отладка программного кода Оформле...,"[разработка_NOUN, отладка_NOUN, программный_AD...",0.0
5,5,5,5,6.001,2,A,Разработка и отладка программного кода,3,A/03.3,Оформление программного кода в соответствии с ...,3,Техник-программист,posts,"['техника_NOUN', 'программист_NOUN']","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Разработка и отладка программного кода Оформле...,"[разработка_NOUN, отладка_NOUN, программный_AD...",0.0
6,6,6,6,6.001,3,A,Разработка и отладка программного кода,3,A/04.3,Работа с системой контроля версий,3,Младший программист,posts,"['младший_ADJF', 'программист_NOUN']","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Разработка и отладка программного кода Работа ...,"[разработка_NOUN, отладка_NOUN, программный_AD...",0.0
7,7,7,7,6.001,3,A,Разработка и отладка программного кода,3,A/04.3,Работа с системой контроля версий,3,Техник-программист,posts,"['техника_NOUN', 'программист_NOUN']","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Разработка и отладка программного кода Работа ...,"[разработка_NOUN, отладка_NOUN, программный_AD...",0.0
8,8,8,8,6.001,4,A,Разработка и отладка программного кода,3,A/05.3,Проверка и отладка программного кода,3,Младший программист,posts,"['младший_ADJF', 'программист_NOUN']","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Разработка и отладка программного кода Проверк...,"[разработка_NOUN, отладка_NOUN, программный_AD...",0.0
9,9,9,9,6.001,4,A,Разработка и отладка программного кода,3,A/05.3,Проверка и отладка программного кода,3,Техник-программист,posts,"['техника_NOUN', 'программист_NOUN']","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Разработка и отладка программного кода Проверк...,"[разработка_NOUN, отладка_NOUN, программный_AD...",0.0


In [350]:
samples_df['prof_name'] = samples_df['labels'].apply(lambda label: 
                                                     'Специалист по интеграции прикладных решений' if label == 21 else
                                                     'Инженер-проектировщик в области связи (телекоммуникаций)' if label == 20 else
                                                     'Разработчик Web и мультимедийных приложений' if label == 19 else
                                                     'Специалист по тестированию в области информационных технологий' if label == 18 else
                                                     'Архитектор программного обеспечения' if label == 17 else
                                                     'Специалист по информационным ресурсам' if label == 16 else
                                                     'Инженер-радиоэлектронщик' if label == 15 else
                                                     'Программист' if label == 14 else
                                                     'Менеджер по продажам информационно-коммуникационных систем' if label == 13 else
                                                     'Системный программист' if label == 12 else
                                                     'Специалист по администрированию сетевых устройств информационно-коммуникационных систем' if label == 11 else
                                                     'Системный администратор информационно-коммуникационных систем' if label == 10 else
                                                     'Специалист по дизайну графических и пользовательских интерфейсов' if label == 9 else
                                                     'Системный аналитик' if label == 8 else
                                                     'Технический писатель (специалист по технической документации в области информационных технологий)' if label == 7 else
                                                     'Руководитель разработки программного обеспечения' if label == 6 else
                                                     'Руководитель проектов в области информационных технологий' if label == 5 else
                                                     'Менеджер по информационным технологиям' if label == 4 else
                                                     'Менеджер продуктов в области информационных технологий' if label == 3 else
                                                     'Специалист по информационным системам' if label == 2 else
                                                     'Администратор баз данных' if label == 1 else
                                                     'NaN')

In [353]:
samples_df['kod_standard'] = samples_df['labels'].apply(lambda label: 
                                                        '6.011' if label == 1 else
                                                        '6.015' if label == 2 else
                                                        '6.012' if label == 3 else
                                                        '6.014' if label == 4 else
                                                        '6.016' if label == 5 else
                                                        '6.017' if label == 6 else
                                                        '6.019' if label == 7 else
                                                        '6.022' if label == 8 else
                                                        '6.025' if label == 9 else
                                                        '6.026' if label == 10 else
                                                        '6.027' if label == 11 else
                                                        '6.028' if label == 12 else
                                                        '6.029' if label == 13 else
                                                        '6.001' if label == 14 else
                                                        '6.005' if label == 15 else
                                                        '6.013' if label == 16 else
                                                        '6.003' if label == 17 else
                                                        '6.004' if label == 18 else
                                                        '6.035' if label == 19 else
                                                        '6.007' if label == 20 else
                                                        '6.041' if label == 21 else
                                                        'NaN')